# Experiment Patch TST

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 114.6 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 901MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q wandb torch torchvision torchaudio -q neuralForecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# Add wandb import - keep the try-except for robustness
try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("wandb not installed. Manual logging to WandB will be skipped.")

# Add NeuralForecast imports
try:
    import neuralforecast
    from neuralforecast import NeuralForecast
    from neuralforecast.models import PatchTST
    from neuralforecast.losses.pytorch import MAE
    NEURALFORECAST_AVAILABLE = True
    print(f"NeuralForecast Version: {neuralforecast.__version__}")
except ImportError:
    NEURALFORECAST_AVAILABLE = False
    print("neuralforecast not installed. Please install with: pip install neuralforecast")


# =====================
# Weights & Biases Setup
# =====================
def setup_wandb(project_name="walmart-patchtst-forecasting-NO-EXOG-manual-log"): # Updated project name
    """Setup Weights & Biases tracking"""
    if not WANDB_AVAILABLE:
        print("wandb not available, skipping logging.")
        return False
    print("🔧 Setting up Weights & Biases...")
    try:
        if wandb.run:
            wandb.finish()

        wandb.init(
            project=project_name,
            name=f"patchtst_no_exog_manual_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            config={
                "model_type": "PatchTST_NoExog", # Indicate no exogenous variables
                "dataset": "Walmart_Sales_Features",
                "framework": "PyTorch",
                "logging_method": "Manual_WandB_log"
            }
        )
        print(f"✅ Wandb initialized successfully!")
        print(f"🔗 Dashboard: {wandb.run.url}")
    except Exception as e:
        print(f"⚠️ Wandb init warning: {e}")
        print("Continuing without wandb tracking...")
        return False
    return True

# =====================
# Data Preprocessing (Your existing pipeline with OUTLIER CHANGE)
# =====================

class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
        elif 'IsHoliday_x' in train_full.columns:
            train_full = train_full.rename(columns={'IsHoliday_x': 'IsHoliday'})
        elif 'IsHoliday_y' in train_full.columns:
            train_full = train_full.rename(columns={'IsHoliday_y': 'IsHoliday'})

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        df_sorted = df.sort_values('Date').reset_index(drop=True)

        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline on training data"""
        print("🔧 Fitting preprocessing pipeline on training data...")

        self.train_data_for_lags = train_data.copy()

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """Transform data using fitted pipeline"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        df = self._create_date_features(df)
        df = self._create_holiday_features(df)
        df = self._encode_categorical_features(df)

        if is_validation:
            df = self._create_lag_features_validation(df)
        else:
            df = self._create_lag_features_training(df)

        if not is_validation:
            df = self._remove_negative_sales(df) # Only remove clearly erroneous negative sales

        df = self._remove_markdown_features(df)
        df = self._remove_redundant_features(df)

        initial_na_count = df.drop(columns=['Weekly_Sales'], errors='ignore').isna().sum().sum()
        if initial_na_count > 0:
            print(f"   ⚠️ Filling {initial_na_count} NaN values in numeric features with 0 (or median/mean).")
            numeric_cols_to_fill = df.select_dtypes(include=np.number).columns.tolist()
            if 'Weekly_Sales' in numeric_cols_to_fill:
                numeric_cols_to_fill.remove('Weekly_Sales')

            for col in numeric_cols_to_fill:
                if df[col].isna().any():
                    median_val = self.train_data_for_lags[col].median() if col in self.train_data_for_lags.columns else 0
                    df[col] = df[col].fillna(median_val if not pd.isna(median_val) else 0)

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)

        df['IsMajorHolidayDerived'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)

        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        return df

    def _create_lag_features_training(self, df):
        return df

    def _create_lag_features_validation(self, df):
        return df

    def _remove_negative_sales(self, df):
        """Remove only negative Weekly_Sales, which are clearly invalid."""
        initial_len = len(df)
        df_clean = df[df['Weekly_Sales'] >= 0].copy()
        removed = initial_len - len(df_clean)
        if removed > 0:
            print(f"   🗑️ Removed {removed:,} records with negative Weekly_Sales from training data.")
        return df_clean

    def _remove_markdown_features(self, df):
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
        return df

    def _remove_redundant_features(self, df):
        return df

    def _encode_categorical_features(self, df):
        df = df.copy()

        if 'Type' in df.columns:
            print(f"   🔧 Encoding Type column using both one-hot and label encoding...")

            type_dummies = pd.get_dummies(df['Type'], prefix='Type', dtype=int)
            type_mapping = {'A': 0, 'B': 1, 'C': 2}
            df['Type_Encoded'] = df['Type'].map(type_mapping)

            for col in type_dummies.columns:
                df[col] = type_dummies[col]

            df = df.drop('Type', axis=1)

            print(f"   ✅ Added both Type_Encoded and {list(type_dummies.columns)}")
        return df


# =====================
# Evaluation Metrics
# =====================

def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

def evaluate_model(y_true, y_pred, is_holiday):
    print("📊 Evaluating model performance...")
    if len(y_true) == 0 or len(y_pred) == 0:
        print("⚠️ No predictions available for evaluation!")
        return {}
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    wmae = calculate_wmae(y_true, y_pred, is_holiday)
    print(f"✅ Evaluation complete! WMAE: {wmae:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return {'WMAE': wmae, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

# =====================
# Main Execution
# =====================

def main():
    print("🚀 Starting PatchTST Experiment (NeuralForecast, Cross-Validation) with More Features")
    if not NEURALFORECAST_AVAILABLE:
        print("❌ NeuralForecast is required. Please install with: pip install neuralforecast")
        return

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Hyperparameters ---
    config = {
        'h': 53,
        'input_size': 52,
        'max_steps': 5000,
        'val_size': 53,
        'batch_size': 256,
        'learning_rate': 1e-3,
        'random_seed': 42,
        'optimizer': 'AdamW',
        'patch_len': 16,
        'stride': 8,
        'revin': True,
        'n_heads': 16,
        'encoder_layers': 3,
        'hidden_size': 128,
        'linear_hidden_size': 256,
        'dropout': 0.2,
        'fc_dropout': 0.2,
        'head_dropout': 0.0,
        'attn_dropout': 0.0
    }

    wandb_enabled = False
    if WANDB_AVAILABLE:
        wandb_enabled = setup_wandb()

    if wandb_enabled:
        if wandb.run:
            wandb.config.update(config)


    # Step 1: Data preprocessing
    pipeline = WalmartPreprocessingPipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)
    pipeline.fit(train_data)

    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data, is_validation=True)

    df_full_dataset = pd.concat([train_processed, val_processed], axis=0)
    df_full_dataset['unique_id'] = df_full_dataset['Store'].astype(str) + '_' + df_full_dataset['Dept'].astype(str)
    df_full_dataset = df_full_dataset.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})

    # *** CHANGE: Remove all exogenous variable lists as PatchTST does not support them ***
    # Leaving them empty will tell NeuralForecast not to pass them to PatchTST
    hist_exog_list = []
    stat_exog_list = []

    panel_df_cols = ['unique_id', 'ds', 'y']

    panel_df = df_full_dataset[panel_df_cols].sort_values(['unique_id', 'ds'])

    min_length = config['input_size'] + config['h']
    series_lengths = panel_df.groupby('unique_id').size()
    long_enough_ids = series_lengths[series_lengths >= min_length].index
    panel_df = panel_df[panel_df['unique_id'].isin(long_enough_ids)]

    if wandb_enabled:
        wandb.log({
            "initial_train_samples_split": len(train_processed),
            "initial_val_samples_split": len(val_processed),
            "overall_data_split_date": str(split_info['split_date']),
            "num_series_after_length_filter": len(long_enough_ids),
            "min_series_length_required": min_length,
            "used_hist_exog_list": hist_exog_list, # Will be empty
            "used_stat_exog_list": stat_exog_list # Will be empty
        })

    series_lengths_final = panel_df.groupby('unique_id').size()
    print(series_lengths_final.describe())
    print(f'Number of series with >= {min_length} points: {(series_lengths_final >= min_length).sum()}')
    print("No Historical or Static Features used in PatchTST (as per NeuralForecast's PatchTST implementation).")


    # Step 2: Model setup
    model = PatchTST(
        h=config['h'],
        input_size=config['input_size'],

        patch_len=config['patch_len'],
        stride=config['stride'],
        revin=config['revin'],
        revin_affine=False,
        revin_subtract_last=False,
        encoder_layers=config['encoder_layers'],
        n_heads=config['n_heads'],
        hidden_size=config['hidden_size'],
        linear_hidden_size=config['linear_hidden_size'],
        dropout=config['dropout'],
        fc_dropout=config['fc_dropout'],
        head_dropout=config['head_dropout'],
        attn_dropout=config['attn_dropout'],
        activation='gelu',

        # *** IMPORTANT CHANGE HERE: Remove all exogenous variable parameters ***
        # Leaving them out or explicitly setting them to [] is required
        # as PatchTST in NeuralForecast does not support any exogenous variables
        # futr_exog_list=[], # No longer needed, as we're not passing any exog
        # hist_exog_list=[], # No longer needed
        # stat_exog_list=[], # No longer needed

        logger=False,

        accelerator=device,
        enable_progress_bar=True,
        val_check_steps=100,
        max_steps=config['max_steps'],
        optimizer=torch.optim.AdamW,
        lr_scheduler=None,
        start_padding_enabled=True,
        batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        loss=MAE(),
        random_seed=config['random_seed']
    )

    fcst = NeuralForecast(models=[model], freq='W')

    # Step 3: Perform cross-validation and get forecasts directly
    print("\n🤖 Performing cross-validation and getting forecasts...")
    forecasts_cv = fcst.cross_validation(
        df=panel_df, # panel_df now only contains unique_id, ds, y
        val_size=config['val_size'],
        test_size=config['h'],
        n_windows=None
    )

    # Step 4: Prepare forecasts for evaluation
    print("\n🔮 Preparing predictions for evaluation...")

    if forecasts_cv is None or forecasts_cv.empty:
        print("⚠️ NeuralForecast cross_validation did not return any predictions. Check data and CV setup.")
        metrics = {}
    else:
        eval_df = forecasts_cv.copy()

        # We still need IsHoliday for WMAE calculation, so merge it back
        # It's no longer an input to the model, but it's needed for the metric
        if 'IsHoliday' not in eval_df.columns:
            eval_df = eval_df.merge(
                df_full_dataset[['unique_id', 'ds', 'IsHoliday']],
                on=['unique_id', 'ds'],
                how='left'
            )
            print("   ❗ Added IsHoliday to eval_df for WMAE calculation (from original data).")

        valid_mask = ~eval_df['PatchTST'].isna() & ~eval_df['y'].isna() & ~eval_df['IsHoliday'].isna()
        eval_df = eval_df[valid_mask].copy()

        if eval_df.empty:
            print("⚠️ No valid predictions for evaluation after filtering. Check data and CV setup.")
            metrics = {}
        else:
            y_true = eval_df['y'].values
            y_pred = eval_df['PatchTST'].values
            is_holiday = eval_df['IsHoliday'].values.astype(bool)

            print(f"Evaluated on {len(y_true):,} validation samples.")
            metrics = evaluate_model(y_true, y_pred, is_holiday)

    # Step 5: Log final metrics to WandB (manually)
    if wandb_enabled:
        if metrics:
            print("✨ Logging final metrics to WandB manually...")
            wandb.log(metrics)
        else:
            print("⚠️ No metrics to log to WandB.")
        wandb.finish()
        print("✅ WandB run finished.")
    else:
        print(f"   ✅ Experiment complete. Metrics: {metrics}")

if __name__ == "__main__":
    main()

NeuralForecast Version: 3.0.2
🚀 Starting PatchTST Experiment (NeuralForecast, Cross-Validation) with More Features
Using device: cuda
🔧 Setting up Weights & Biases...


initial_train_samples_split,▁
initial_val_samples_split,▁
min_series_length_required,▁
num_series_after_length_filter,▁
initial_train_samples_split,336266
initial_val_samples_split,84314
min_series_length_required,105
num_series_after_length_filter,2862
overall_data_split_date,2012-04-13 00:00:00


✅ Wandb initialized successfully!
🔗 Dashboard: https://wandb.ai/konstantine25b-free-university-of-tbilisi-/walmart-patchtst-forecasting-NO-EXOG-manual-log/runs/sje09f04
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training data...
   🔧 Encoding Type column using both one-hot and label encoding...
   ✅ Added both Type_Encoded and ['Type_A', 'Type_B', 'Type_C']
   🗑️ Removed 990 records with negative W

INFO:lightning_fabric.utilities.seed:Seed set to 42


count    2862.000000
mean      141.846261
std         5.113543
min       105.000000
25%       143.000000
50%       143.000000
75%       143.000000
max       143.000000
dtype: float64
Number of series with >= 105 points: 2862
No Historical or Static Features used in PatchTST (as per NeuralForecast's PatchTST implementation).

🤖 Performing cross-validation and getting forecasts...


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]


🔮 Preparing predictions for evaluation...
   ❗ Added IsHoliday to eval_df for WMAE calculation (from original data).
Evaluated on 151,686 validation samples.
📊 Evaluating model performance...
✅ Evaluation complete! WMAE: 4311.47, MAE: 4228.89, RMSE: 9884.75, R²: 0.8218
✨ Logging final metrics to WandB manually...


MAE,▁
R2,▁
RMSE,▁
WMAE,▁
initial_train_samples_split,▁
initial_val_samples_split,▁
min_series_length_required,▁
num_series_after_length_filter,▁
MAE,4228.88749
R2,0.82175
RMSE,9884.74695


✅ WandB run finished.
